In [1]:
#libraries will be imported were ever necessary

In [2]:
from pyspark.sql import SparkSession

In [3]:
#creating basic Saprk session
spark=SparkSession.builder.appName('churn_modelling').getOrCreate()

In [4]:
#kaggle dataset link : https://www.kaggle.com/shrutimechlearn/churn-modelling

In [5]:
df=spark.read.csv('D://M. Tech in Data Science & Machine Learning//Big Data Analytics//Sem_Prep//Churn_Modelling//Churn_Modelling.csv', header=True,inferSchema=True)
df.show(2,False)

+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname |CreditScore|Geography|Gender|Age|Tenure|Balance |NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|1        |15634602  |Hargrave|619        |France   |Female|42 |2     |0.0     |1            |1        |1             |101348.88      |1     |
|2        |15647311  |Hill    |608        |Spain    |Female|41 |1     |83807.86|1            |0        |1             |112542.58      |0     |
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
only showing top 2 rows



In [6]:
df.count()#total records

10000

In [7]:
len(df.columns)#total length of the columns

14

In [8]:
df.printSchema()#column summary

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [9]:
for i, t in df.dtypes:#column and its datatypes
    print('column Name:', i, ' ','column datatype:',t)

column Name: RowNumber   column datatype: int
column Name: CustomerId   column datatype: int
column Name: Surname   column datatype: string
column Name: CreditScore   column datatype: int
column Name: Geography   column datatype: string
column Name: Gender   column datatype: string
column Name: Age   column datatype: int
column Name: Tenure   column datatype: int
column Name: Balance   column datatype: double
column Name: NumOfProducts   column datatype: int
column Name: HasCrCard   column datatype: int
column Name: IsActiveMember   column datatype: int
column Name: EstimatedSalary   column datatype: double
column Name: Exited   column datatype: int


In [10]:
df.toPandas().describe()#stastical summary

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [11]:
from pyspark.sql.functions import isnan, count, col, when

In [12]:
#checking for null values
df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|        0|         0|      0|          0|        0|     0|  0|     0|      0|            0|        0|             0|              0|     0|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+



In [13]:
#There are no null values in the dataset

In [14]:
df.columns

['RowNumber',
 'CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [15]:
df=df.drop('RowNumber','CustomerId','Surname')

In [16]:
#Bucketizer

In [17]:
df.groupby('Age').count().show()

+---+-----+
|Age|count|
+---+-----+
| 31|  404|
| 85|    1|
| 65|   18|
| 53|   74|
| 78|    5|
| 34|  447|
| 81|    4|
| 28|  273|
| 76|   11|
| 27|  209|
| 26|  200|
| 44|  257|
| 22|   84|
| 47|  175|
| 52|  102|
| 40|  432|
| 20|   40|
| 57|   75|
| 54|   84|
| 48|  168|
+---+-----+
only showing top 20 rows



In [18]:
df.agg({'Age':'max'}).show()

+--------+
|max(Age)|
+--------+
|      92|
+--------+



In [19]:
df.agg({'Age':'min'}).show()

+--------+
|min(Age)|
+--------+
|      18|
+--------+



In [20]:
#Age groups are between 18 and 92, we use age group

In [21]:
from pyspark.ml.feature import Bucketizer

In [22]:
split=[18,30,50,70,92]
bucketizer=Bucketizer(splits=split,inputCol='Age',outputCol='Age_group')
df1=bucketizer.transform(df)

In [23]:
df1.groupby('Age_group').count().show()

+---------+-----+
|Age_group|count|
+---------+-----+
|      0.0| 1641|
|      1.0| 6964|
|      3.0|  151|
|      2.0| 1244|
+---------+-----+



In [24]:
#String Indexer

In [25]:
Indexers=[]
for i, t in df.dtypes:#column and its datatypes
    if t=='string':
        print('column Name:', i, ' ','column datatype:',t)
        Indexers.append(i)

column Name: Geography   column datatype: string
column Name: Gender   column datatype: string


In [26]:
Indexers

['Geography', 'Gender']

In [27]:
from pyspark.ml.feature import StringIndexer

In [28]:
indexing=StringIndexer(inputCols=['Geography', 'Gender','Exited'],
                  outputCols=['Geography_index', 'Gender_index','label'])
df2=indexing.fit(df1).transform(df1)

In [29]:
df2=df2.drop('Geography', 'Gender','Exited')
df2.show(2)

+-----------+---+------+--------+-------------+---------+--------------+---------------+---------+---------------+------------+-----+
|CreditScore|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Age_group|Geography_index|Gender_index|label|
+-----------+---+------+--------+-------------+---------+--------------+---------------+---------+---------------+------------+-----+
|        619| 42|     2|     0.0|            1|        1|             1|      101348.88|      1.0|            0.0|         1.0|  1.0|
|        608| 41|     1|83807.86|            1|        0|             1|      112542.58|      1.0|            2.0|         1.0|  0.0|
+-----------+---+------+--------+-------------+---------+--------------+---------------+---------+---------------+------------+-----+
only showing top 2 rows



In [30]:
df2.groupby('label').count().show()#reanmed the colum from Exited to label

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 7963|
|  1.0| 2037|
+-----+-----+



In [31]:
#Vector Assembler

In [32]:
from pyspark.ml.feature import VectorAssembler

In [33]:
df2.columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Age_group',
 'Geography_index',
 'Gender_index',
 'label']

In [34]:
assembler=VectorAssembler(inputCols=['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Age_group','Geography_index','Gender_index'], outputCol='features')
df3=assembler.transform(df2)

In [35]:
df3.select('features','label').show(3,False)

+-----------------------------------------------------------+-----+
|features                                                   |label|
+-----------------------------------------------------------+-----+
|[619.0,42.0,2.0,0.0,1.0,1.0,1.0,101348.88,1.0,0.0,1.0]     |1.0  |
|[608.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,1.0,2.0,1.0]|0.0  |
|[502.0,42.0,8.0,159660.8,3.0,1.0,0.0,113931.57,1.0,0.0,1.0]|1.0  |
+-----------------------------------------------------------+-----+
only showing top 3 rows



In [36]:
#Vector Indexer

In [37]:
from pyspark.ml.feature import VectorIndexer

In [38]:
vector_indexer=VectorIndexer(inputCol='features', outputCol='indexed_features')
df4=vector_indexer.fit(df3).transform(df3)

In [39]:
df4.select('features','indexed_features','label').show(2)

+--------------------+--------------------+-----+
|            features|    indexed_features|label|
+--------------------+--------------------+-----+
|[619.0,42.0,2.0,0...|[619.0,42.0,2.0,0...|  1.0|
|[608.0,41.0,1.0,8...|[608.0,41.0,1.0,8...|  0.0|
+--------------------+--------------------+-----+
only showing top 2 rows



In [40]:
#Train Test Split

In [41]:
train1, test1 = df4.randomSplit([0.70, 0.30], 2)

In [42]:
train1.count(),test1.count()

(7049, 2951)

In [43]:
#Model Building

In [44]:
#1. LogisticRegression

In [45]:
from pyspark.ml.classification import LogisticRegression

In [46]:
lr=LogisticRegression(featuresCol='features',labelCol='label')
lr_model=lr.fit(train1)
lr_model_summary=lr_model.summary

In [47]:
print('Accuracy:',lr_model_summary.accuracy)
print('Precision:',lr_model_summary.weightedPrecision)
print('Recall:',lr_model_summary.weightedRecall)
print('F1 Score:',lr_model_summary.weightedFMeasure())
print('Area under the curve :',lr_model_summary.areaUnderROC)

Accuracy: 0.8043694141012909
Precision: 0.766111004227326
Recall: 0.8043694141012909
F1 Score: 0.7599154444511099
Area under the curve : 0.7591301059796025


In [48]:
#2. Decision Tree Classifier

In [49]:
from pyspark.ml.classification import DecisionTreeClassifier

In [50]:
dt=DecisionTreeClassifier(featuresCol='indexed_features', labelCol='label')
dt_model=dt.fit(train1)
dt_prediction=dt_model.transform(test1)

In [51]:
dt_prediction.columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Age_group',
 'Geography_index',
 'Gender_index',
 'label',
 'features',
 'indexed_features',
 'rawPrediction',
 'probability',
 'prediction']

In [52]:
dt_prediction.select('rawPrediction','probability','prediction','label').show(2,False)

+-------------+----------------------------------------+----------+-----+
|rawPrediction|probability                             |prediction|label|
+-------------+----------------------------------------+----------+-----+
|[17.0,173.0] |[0.08947368421052632,0.9105263157894737]|1.0       |1.0  |
|[6.0,61.0]   |[0.08955223880597014,0.9104477611940298]|1.0       |1.0  |
+-------------+----------------------------------------+----------+-----+
only showing top 2 rows



In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [54]:
evaluator=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label')
print('Accuracy:',evaluator.evaluate(dt_prediction, {evaluator.metricName: "accuracy"}))
print('f1:',evaluator.evaluate(dt_prediction, {evaluator.metricName: "f1"}))
print('Precision:',evaluator.evaluate(dt_prediction, {evaluator.metricName: "weightedPrecision"}))
print('Recall:',evaluator.evaluate(dt_prediction, {evaluator.metricName: "weightedRecall"}))
print('Area under the ROC curve = ',evaluator.evaluate(dt_prediction))

Accuracy: 0.8505591324974585
f1: 0.8365268722205028
Precision: 0.8395040430562379
Recall: 0.8505591324974585
Area under the ROC curve =  0.8365268722205028


In [55]:
#3. Random Forest Classifier

In [56]:
from pyspark.ml.classification import RandomForestClassifier

In [57]:
rf = RandomForestClassifier(featuresCol='indexed_features',labelCol='label')
rf_model= rf.fit(train1)
rf_prediction = rf_model.transform(test1)

In [58]:
rf_prediction.select('rawPrediction','probability','prediction','label').show(2,False)

+--------------------------------------+---------------------------------------+----------+-----+
|rawPrediction                         |probability                            |prediction|label|
+--------------------------------------+---------------------------------------+----------+-----+
|[7.692143325657921,12.30785667434208] |[0.38460716628289604,0.615392833717104]|1.0       |1.0  |
|[6.818034242892532,13.181965757107468]|[0.3409017121446266,0.6590982878553734]|1.0       |1.0  |
+--------------------------------------+---------------------------------------+----------+-----+
only showing top 2 rows



In [59]:
evaluator=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label')
print('Accuracy:',evaluator.evaluate(rf_prediction, {evaluator.metricName: "accuracy"}))
print('f1:',evaluator.evaluate(rf_prediction, {evaluator.metricName: "f1"}))
print('Precision:',evaluator.evaluate(rf_prediction, {evaluator.metricName: "weightedPrecision"}))
print('Recall:',evaluator.evaluate(rf_prediction, {evaluator.metricName: "weightedRecall"}))
print('Area under the ROC curve = ',evaluator.evaluate(rf_prediction))

Accuracy: 0.8542866824805151
f1: 0.8300452843906327
Precision: 0.8543755642029085
Recall: 0.8542866824805151
Area under the ROC curve =  0.8300452843906327


In [61]:
#pipeline for logistic Regression

In [65]:
train2, test2 =df.randomSplit([0.7,0.3],2)

In [66]:
from pyspark.ml import Pipeline

In [67]:
lr_pipeline=Pipeline(stages = [bucketizer,indexing,assembler,lr])
lr_pipeline_model=lr_pipeline.fit(train2)
lr_pipeline_prediciton=lr_pipeline_model.transform(test2)

In [68]:
evaluator=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label')
print('Accuracy:',evaluator.evaluate(lr_pipeline_prediciton, {evaluator.metricName: "accuracy"}))
print('f1:',evaluator.evaluate(lr_pipeline_prediciton, {evaluator.metricName: "f1"}))
print('Precision:',evaluator.evaluate(lr_pipeline_prediciton, {evaluator.metricName: "weightedPrecision"}))
print('Recall:',evaluator.evaluate(lr_pipeline_prediciton, {evaluator.metricName: "weightedRecall"}))
print('Area under the ROC curve = ',evaluator.evaluate(rf_prediction))

Accuracy: 0.8085394781430024
f1: 0.7655328517940844
Precision: 0.7794328086348744
Recall: 0.8085394781430024
Area under the ROC curve =  0.8300452843906327


In [60]:
#------------------------------------------------END--------------------------------------------#